In [1]:
import pandas as pd
data = pd.read_csv("facebook.csv")
print(data.head())



   0  1
0  0  2
1  0  3
2  0  4
3  0  5
4  0  6


In [2]:
import networkx as nx
import numpy as np
G=nx.from_pandas_edgelist(data, "0", "1") 
print(len(G))
print(nx.is_connected(G))


4039
True


In [3]:
from collections import defaultdict
communities = nx.algorithms.community.label_propagation.label_propagation_communities(G)
communities = list(communities)
communities.sort(key=len, reverse=True)

#for i, community in enumerate(communities):
#     print(f'Community {i+1}:', len(community))

    
k = 9
final_communities = []
cluster_count = 0
node_cluster_map = defaultdict(int)

for community in communities:
    if cluster_count < k:
        final_communities.append(community)
        for node in community:
            node_cluster_map[node] = cluster_count
        cluster_count += 1
    else:
        for node in community:
            node_cluster_map[node] = -1
        #break

cluster_labels = {
    0: "Politics",
    1: "Sports",
    2: "Entertainment",
    3: "Technology",
    4: "Business",
    5: "Science",
    6: "Education",
    7: "Health",
    8: "Art"
}


connections = []
for node in G.nodes():
    neighbors = list(G.neighbors(node))
    connections.append(neighbors)


df = pd.DataFrame({'Node': G.nodes(), 'Connections': connections})
df['Label'] = [cluster_labels.get(node_cluster_map.get(node, -1), 'Other') for node in G.nodes()]
#df['Label'] = [cluster_labels[node_cluster_map[node]] for node in G.nodes()]
df = df.sort_values('Node')
df = df.reset_index(drop=True)
print(df)


#see neighbors of 0 and their labels
# node0_data = df.loc[df['Node'] == 0]
# node0_connections = node0_data['Connections'].iloc[0]
# node0_labels = df.loc[df['Node'].isin(node0_connections), 'Label'].tolist()
# label_counts = defaultdict(int)
# for neighbor in node0_connections:
#     label = df.loc[df['Node'] == neighbor, 'Label'].iloc[0]
#     label_counts[label] += 1
# print("Label counts:")
# for label, count in label_counts.items():
#     print(f"{label}: {count}")


cluster_lists = [[] for _ in range(k+1)]
for node, cluster_id in node_cluster_map.items():
    cluster_lists[cluster_id].append(node)


for i, cluster_nodes in enumerate(cluster_lists):
    print(f'Cluster {i+1}:', len(cluster_nodes))

      Node                                        Connections      Label
0        0  [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 1...  Education
1        1  [48, 53, 54, 73, 88, 92, 119, 126, 133, 194, 2...  Education
2        2    [0, 20, 115, 116, 149, 226, 312, 326, 333, 343]      Other
3        3  [0, 9, 25, 26, 67, 72, 85, 122, 142, 170, 188,...  Education
4        4    [0, 78, 152, 181, 195, 218, 273, 275, 306, 328]      Other
...    ...                                                ...        ...
4034  4034                                       [3980, 4023]        Art
4035  4035                                             [3980]        Art
4036  4036                                       [3980, 3999]        Art
4037  4037                           [3980, 3982, 4014, 4020]        Art
4038  4038  [3980, 3989, 4004, 4013, 4014, 4020, 4023, 402...        Art

[4039 rows x 3 columns]
Cluster 1: 1030
Cluster 2: 753
Cluster 3: 547
Cluster 4: 469
Cluster 5: 226
Cluster 6: 215
Cluster 

In [4]:
# import itertools
# from networkx.algorithms.community.centrality import girvan_newman
# k = 2
# comp = girvan_newman(G)
# limited = itertools.takewhile(lambda c: len(c) <= k, comp)
# for communities in limited:
#     print(c for c in communities)

In [12]:
import random
import networkx as nx
import pandas as pd

import networkx as nx
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()

class PersonalizedPageRank:
    def __init__(self, graph, dataframe):
        self.graph = graph
        self.dataframe = dataframe

    def give_input(self, label):
        # Select nodes with the specified label for personalization
        nodes_with_label = self.dataframe[self.dataframe['Label'] == label]['Node'].tolist()
        return nodes_with_label

    def fit(self, label, alpha=0.15, max_iter=100, tol=1e-6): ####This is the method implemented as in the youtube lecture from standford
        # Personalized PageRank algorithm
        nodes_with_label = self.give_input(label)
        num_nodes = len(self.graph.nodes)
        num_nodes_label = len(nodes_with_label)

        if num_nodes_label == 0: ###If no personalization 
            personalization = dict.fromkeys(self.graph.nodes, 1/num_nodes)
        else:
            personalization = dict.fromkeys(self.graph.nodes, 0)
            personalization.update({node: 1 / len(nodes_with_label) for node in nodes_with_label}) ##personalization given by topic

        # Initialize the importance scores
        importance_scores = dict.fromkeys(self.graph.nodes, 1 / num_nodes)

        # Iterate until convergence or maximum iterations
        for _ in range(max_iter):
            prev_scores = importance_scores.copy()
            importance_scores = dict.fromkeys(self.graph.nodes, 0) #reset current scores
            print('prev',prev_scores)

            for node in self.graph.nodes: ## do updates for all nodes
                if node in nodes_with_label:## if it has the given topic then add that importance  (since random teleportation is now between one of the nodes with that topic)
                    importance_scores[node]+=(1 - alpha) / num_nodes_label
                for neighbor in self.graph.neighbors(node): ## add importance from every neighbour
                    importance_scores[node] += alpha * prev_scores[neighbor] / self.graph.degree[neighbor]

            # Check for convergence
            max_diff = max(abs(importance_scores[node] - prev_scores[node]) for node in self.graph.nodes)
            if max_diff < tol:
                break
        
        # Add an 'Importance Rank' column to the DataFrame
        self.dataframe['Importance Rank'] = self.dataframe['Node'].map(importance_scores)
        self.dataframe['Importance Rank']=self.dataframe['Importance Rank']/ np.sum(self.dataframe['Importance Rank'].to_numpy()) ##scale the importance to make it stochastic

    def fit2(self, label, alpha=0.85, max_iter=100, tol=1e-6): ## This is just for checking using netwrokx implementation ###Posibil sa o sterg cand trimitem tema.  
        # Personalized PageRank algorithm using NetworkX's pagerank

        nodes_with_label = self.give_input(label)
        personalization = dict.fromkeys(self.graph.nodes, 0)
        personalization.update({node: 1 / len(nodes_with_label) for node in nodes_with_label})

        # Compute the PageRank scores using NetworkX
        pr = nx.pagerank(self.graph, alpha=alpha, personalization=personalization,
                         max_iter=max_iter, tol=tol)

        # Add an 'Importance Rank' column to the DataFrame
        self.dataframe['Importance Rank'] = self.dataframe['Node'].map(pr)
    

  
    def return_important_pages(self):
        # Sort the DataFrame by the 'Importance Rank' column in descending order
        sorted_dataframe = self.dataframe.sort_values('Importance Rank', ascending=False)

        return sorted_dataframe




ppr = PersonalizedPageRank(G, df)

# Use the fit method to compute the personalized PageRank
label = "Sports"  # Replace with the label you want to personalize for
ppr.fit2(label)

# Get the important pages sorted by their importance rank
sorted_df = ppr.return_important_pages()
print(sorted_df.head(10))




      Node                                        Connections   Label   
1912  1912  [58, 136, 428, 563, 1465, 1577, 1718, 1913, 19...  Sports  \
2047  2047  [1465, 1577, 1912, 1920, 1923, 1926, 1930, 193...  Sports   
2313  2313  [1912, 1926, 1930, 1934, 1960, 1964, 1965, 197...  Sports   
2543  2543  [136, 1465, 1577, 1718, 1912, 1917, 1918, 1920...  Sports   
2347  2347  [1465, 1577, 1912, 1917, 1918, 1920, 1925, 192...  Sports   
2384  2384  [136, 1465, 1577, 1912, 1913, 1916, 1920, 1926...  Sports   
2133  2133  [136, 1577, 1718, 1912, 1916, 1923, 1926, 1940...  Sports   
2289  2289  [136, 1465, 1577, 1718, 1912, 1916, 1923, 1926...  Sports   
2111  2111  [136, 1465, 1577, 1718, 1912, 1916, 1920, 1926...  Sports   
2199  2199  [136, 1465, 1577, 1718, 1912, 1916, 1920, 1926...  Sports   

      Importance Rank  
1912         0.019981  
2047         0.004341  
2313         0.003882  
2543         0.003525  
2347         0.003314  
2384         0.002995  
2133         0.002987  
2289

In [11]:
##This is inspired from geekforgeeks  (https://www.geeksforgeeks.org/page-rank-algorithm-implementation/). Aproape identica cu ce e pe site 

def pagerank(G=G, alpha=0.85, personalization=None, max_iter=100, tol=1.0e-6, nstart=None, weight='weight', dangling=None):
    if len(G) == 0:
        return {}
   
    ## make it suitable for py libraries
    if not G.is_directed():
        D = G.to_directed()
    else:
        D = G
    print('D',D)
  
    W = nx.stochastic_graph(D, weight=weight)
    print('W',W[0])
    N = W.number_of_nodes()
    print('dangling nodes: ', [n for n in W if W.out_degree(n, weight=weight) == 0.0])
  

###if we use a sensible initializing method for nstart
    if nstart is None:
        x = dict.fromkeys(W, 1.0 / N) ###starting importance vector
    else:
        # Normalized nstart vector
        s = float(sum(nstart.values()))
        x = dict((k, v / s) for k, v in nstart.items())
###########


    if personalization is None:
  
        # Assign uniform personalization vector if not given
        p = dict.fromkeys(W, 1.0 / N) ##personalization keys
    else: ##if there is personalization
        s = float(sum(personalization.values()))
        p = dict((k, v / s) for k, v in personalization.items()) ##scale the given personalization to make it stochastic


  
    # power iteration: make up to max_iter iterations
    for _ in range(max_iter):
        xlast = x ##store previous x 
        x = dict.fromkeys(xlast.keys(), 0)
        for n in x: ## for all indeces of nodes 
            for nbr in W[n]: #for all neighbours in the graph (nodes that have in links from node with index n )
                x[nbr] += alpha * xlast[n] * W[n][nbr][weight] ## importance given by in link from node n. In our case weight is uniform over all edges 
            x[n] +=  (1.0 - alpha) * p[n] ## importance added by teleportation 
  
        # check convergence, l1 norm
        err = sum([abs(x[n] - xlast[n]) for n in x])
        if err < N*tol:
            return sorted(x.items(), key= lambda item : item[1],reverse= True)
    print('pagerank: power iteration failed to converge in {max_iter} initerations.' )
    return sorted(x.items(), key= lambda item : item[1])



## use the function for some label
label='Sports'
nodes_with_label = df[df['Label'] == label]['Node'].tolist()
if len(nodes_with_label) == 0: ###If no personalization 
    personalization = dict.fromkeys(G.nodes, 1/len(G.nodes))
else:
    personalization = dict.fromkeys(G.nodes, 0)
    personalization.update({node: 1 / len(nodes_with_label) for node in nodes_with_label})
pagerank(personalization=personalization)


D DiGraph with 4039 nodes and 176466 edges
W {2: {'weight': 0.002890173410404624}, 3: {'weight': 0.002890173410404624}, 4: {'weight': 0.002890173410404624}, 5: {'weight': 0.002890173410404624}, 6: {'weight': 0.002890173410404624}, 7: {'weight': 0.002890173410404624}, 8: {'weight': 0.002890173410404624}, 9: {'weight': 0.002890173410404624}, 10: {'weight': 0.002890173410404624}, 11: {'weight': 0.002890173410404624}, 12: {'weight': 0.002890173410404624}, 13: {'weight': 0.002890173410404624}, 14: {'weight': 0.002890173410404624}, 15: {'weight': 0.002890173410404624}, 16: {'weight': 0.002890173410404624}, 17: {'weight': 0.002890173410404624}, 18: {'weight': 0.002890173410404624}, 19: {'weight': 0.002890173410404624}, 20: {'weight': 0.002890173410404624}, 21: {'weight': 0.002890173410404624}, 22: {'weight': 0.002890173410404624}, 23: {'weight': 0.002890173410404624}, 24: {'weight': 0.002890173410404624}, 25: {'weight': 0.002890173410404624}, 26: {'weight': 0.002890173410404624}, 27: {'weight

[(1912, 0.019980945165762796),
 (2047, 0.004341258750905215),
 (2313, 0.0038815562941436374),
 (2543, 0.0035248062250689692),
 (2347, 0.0033139047142973655),
 (2384, 0.0029946522961430922),
 (2133, 0.0029865270705797444),
 (2289, 0.0029252152768327595),
 (2111, 0.002878819095435181),
 (2199, 0.0028238595485665097),
 (1964, 0.00280758556045004),
 (1941, 0.002797283296995345),
 (2081, 0.002674831020921262),
 (2283, 0.002671588538780649),
 (2328, 0.0026391120470310604),
 (2007, 0.002552374393648221),
 (2054, 0.0025409994806861055),
 (2598, 0.002531137125417736),
 (2333, 0.0025236235338083994),
 (2117, 0.0025072491990927592),
 (1577, 0.0025019546990159946),
 (2087, 0.0024632521567445652),
 (2282, 0.0024552578190894275),
 (2266, 0.0024319440832365185),
 (2126, 0.0023878155184044124),
 (1926, 0.0023870589458886544),
 (2642, 0.0023380599712434945),
 (1985, 0.0023280816651269015),
 (2336, 0.0023268214236614967),
 (2032, 0.0022997055422281066),
 (2142, 0.0022891215539287252),
 (2268, 0.00228471